In [7]:
import pandas as pd

train_df = pd.read_csv("prediction_challenge_train.csv")
test_df = pd.read_csv("prediction_challenge_test.csv")
airport_df = pd.read_csv("airport_country_code_mapping.csv")


train_df = pd.merge(train_df, airport_df, on="Airport Country Code", how="left")
test_df = pd.merge(test_df, airport_df, on="Airport Country Code", how="left")

train_df["Departure Date"] = pd.to_datetime(train_df["Departure Date"], errors="coerce")
train_df["Dep Day"] = train_df["Departure Date"].dt.day
train_df["Dep Month"] = train_df["Departure Date"].dt.month
train_df["Dep Year"] = train_df["Departure Date"].dt.year

test_df["Departure Date"] = pd.to_datetime(test_df["Departure Date"], errors="coerce")
test_df["Dep Day"] = test_df["Departure Date"].dt.day
test_df["Dep Month"] = test_df["Departure Date"].dt.month
test_df["Dep Year"] = test_df["Departure Date"].dt.year


def apply_remaining_rules(row):
    if row["Dep Month"] == 1 or row["Dep Month"] == 12:
        return "Yes"
    
    if (row["Gender"] == "Female" and (row["Ticket Price"] % 4 == 0)):
        return "Yes"
    
    if row["Country Name"] == row["Nationality"]:
        return "Yes"
    
    return "No"


age_mask_train = (train_df["Age"] < 5) | (train_df["Age"] > 85)
train_df.loc[age_mask_train, "Predicted"] = "Yes"

remaining_mask_train = ~age_mask_train
train_df.loc[remaining_mask_train, "Predicted"] = train_df[remaining_mask_train].apply(apply_remaining_rules, axis=1)

accuracy = (train_df["Predicted"] == train_df["Eligible_For_Discount"]).mean()
print("Training accuracy:", accuracy)


age_mask_test = (test_df["Age"] < 5) | (test_df["Age"] > 85)
test_df["Eligible_For_Discount"] = test_df["Eligible_For_Discount"].astype(object)
test_df.loc[age_mask_test, "Eligible_For_Discount"] = "Yes"

remaining_mask_test = ~age_mask_test
test_df.loc[remaining_mask_test, "Eligible_For_Discount"] = test_df[remaining_mask_test].apply(apply_remaining_rules, axis=1)

test_df.to_csv("prediction_challenge_test_predictions.csv", index=False)

submission = test_df[['ID', 'Eligible_For_Discount']]
submission.to_csv('my_predictions_saved.csv', index=False)
print("Predictions saved to my_predictions_saved.csv")

Training accuracy: 1.0
Predictions saved to my_predictions_saved.csv
